### 객체 감지

In [5]:
import cv2
import numpy as np
import requests
import platform
from io import BytesIO
from PIL import Image, ImageDraw, ImageFont

weights_path = "yolo/yolov3.weights"
config_path = "yolo/yolov3.cfg"
names_path = "yolo/coco.names"

with open(names_path, "r", encoding="utf-8") as coco_file :
    label_list = coco_file.read().strip().split('\n')

net = cv2.dnn.readNet(weights_path, config_path)

def random_color() :
  import random
  return (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255)) 

def get_font() :
  import platform
  font_size = 20
  try:
      if platform.system() == "Windows":
          # 윈도우: 맑은 고딕
          return ImageFont.truetype("malgun.ttf", font_size)
      elif platform.system() == "Darwin":  # macOS
          # 맥: 애플 고딕
          return ImageFont.truetype("AppleGothic.ttf", font_size)
      else:  # Linux 등
          # 기본 폰트 (한글 지원 안 될 수 있음)
          return ImageFont.load_default(size=font_size)
  except IOError:
      # 지정한 폰트 파일이 없을 경우 PIL 기본 폰트 사용
      return ImageFont.load_default()


def detect_objects(image_array) :

    image_array = image_array.copy()

    image_height, image_width = image_array.shape[:2]

    blob = cv2.dnn.blobFromImage(image_array, 1/255.0, (416, 416), swapRB=True, crop=False)

    net.setInput(blob)

    out_layer_list = net.getUnconnectedOutLayersNames()
    detection_list = net.forward(out_layer_list)

    bounding_box_list = list()
    confidence_list = list()
    label_index_list = list()

    for prediction_list in detection_list :
        color = random_color()
        for prediction in prediction_list :
            bounding_box = prediction[:4] * np.array([image_width, image_height, image_width, image_height])
            center_x, center_y, w, h = bounding_box.astype('int')
            label_score_list = prediction[5:]

            x = int(center_x - w/2)
            y = int(center_y - h/2)

            label_index = np.argmax(label_score_list)
            confidence = label_score_list[label_index]

            if confidence > 0 :
                bounding_box_list.append([x, y, w ,h])
                confidence_list.append(confidence)
                label_index_list.append(label_index)

    # print(len(bounding_box_list), len(confidence_list), len(label_index_list))

    extracted_index_list = cv2.dnn.NMSBoxes(bounding_box_list, confidence_list, 0.7, 0.3)
    extracted_index_list

    for extracted_index in extracted_index_list :
        color = random_color()
        x, y, w, h = bounding_box_list[extracted_index]
        confidence = confidence_list[extracted_index]
        label_index = label_index_list[extracted_index]
        label_text = label_list[label_index]

        # cv2.rectangle(image_array, (x, y), (x + w, y + h), color, 2)
        cv2.rectangle(image_array, (x, y), (x + w, y + h), (0, 255, 0), 2)
        # cv2.putText(image_array, f"{label_text} {confidence:.2f}", (x, y + 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
        cv2.putText(image_array, f"{label_text} {confidence:.2f}", (x, y + 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

    return image_array

# image_array = cv2.cvtColor(image_array, cv2.COLOR_BGR2RGB)
# print(image_array)
# Image.fromarray(image_array)
# Image.open(BytesIO(image_data))

### dotenv 관련

In [6]:
import os
from dotenv import load_dotenv

load_dotenv()

openai_api_key = os.getenv("AZURE_OAI_KEY")
openai_endpoint = os.getenv("AZURE_OAI_ENDPOINT")

### OpenAI

In [7]:
import io
import base64
from PIL import Image, ImageDraw, ImageFont

def request_gpt(image_array) :

    image = Image.fromarray(image_array)
    byte_image = io.BytesIO()
    image.save(byte_image, format='png')
    base64_image = base64.b64encode(byte_image.getvalue()).decode('utf-8')

    endpoint = openai_endpoint

    headers = {
        "Api-Key" : openai_api_key
    }

    body = {
        "messages": [
            {
                "role": "system",
                "content": [
                    {
                        "type": "text",
                        "text": """
                        당신은 YOLO 객체 감지 모델의 분석 결과를 전문적으로 해설하는 AI 분석가입니다.
                        제공된 이미지와 함께 전달된 객체 감지 결과(감지 확률, 객체 종류, 위치 등)를 기반으로 하여, 각 감지된 물체에 대해 가장 상세하고 정확하게 설명해야 합니다.
                        모든 응답은 한국어로 작성해야 합니다.
                        """
                    }
                ]
            },
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": """
                        첨부된 사진은 YOLO 모델을 통해 분석되었으며, 해당 모델이 감지한 **모든 물체**에 대한 정보를 바탕으로 상세 분석 보고서를 작성해 주세요.

                        **[요청 사항]**
                        1.  **감지된 물체 목록**을 '객체 종류', '감지 확률', '이미지 내 위치(바운딩 박스 좌표/대략적 위치)'를 요약해 주세요.
                        2.  **각 물체별로** 다음 내용을 포함하여 **자세한 설명**을 항목별로 추가해 주세요.
                            * 물체의 특징 (색상, 상태, 용도 등)
                            * 물체의 예상되는 역할 또는 주변 환경과의 관계
                        3.  **감지되지 않은 영역이나 배경에 대한 설명은 일체 포함하지 마세요.**
                        """
                    },
                    {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/png;base64,{base64_image}"
                    }
                    }
                ]
            }
        ],
        "temperature": 0.7,
        "top_p": 0.95,
        "frequency_penalty": 0,
        "presence_penalty": 0,
        "max_tokens": 16384,
        "stop": None,
        "stream": False
    }
    
    response = requests.post(endpoint, headers=headers, json=body)

    if response.status_code != 200 :
        return None
    
    response_json = response.json()
    content = response_json['choices'][0]['message']['content']

    return content


TEST_IMAGE_URL = "https://img.freepik.com/free-photo/close-up-people-preparing-christmas-dinner_23-2149144979.jpg"
response = requests.get(TEST_IMAGE_URL)
image_data = response.content
image_np = np.frombuffer(image_data, np.uint8)
image_array = cv2.imdecode(image_np, cv2.IMREAD_COLOR)

request_gpt(image_array)

'### 1. 감지된 물체 목록\n\n| 객체 종류       | 감지 확률 | 이미지 내 위치                          |\n|------------------|-----------|-----------------------------------------|\n| 테이블           | 0.95      | 중앙 하단 영역                         |\n| 음료수 (와인 잔) | 0.92      | 테이블 좌측 및 우측 영역               |\n| 음식 (접시)     | 0.90      | 테이블 중앙 및 우측 영역               |\n| 장식 (크리스마스 트리) | 0.85 | 배경 좌측 상단                          |\n\n---\n\n### 2. 각 물체별 상세 설명\n\n#### 1. 테이블\n- **물체의 특징**: 테이블은 넓고 평평한 표면으로, 여러 가지 음식과 음료를 담고 있습니다. 일반적으로 나무 또는 금속 소재로 만들어질 수 있으며, 다양한 색상과 질감을 가질 수 있습니다.\n- **물체의 예상되는 역할**: 테이블은 식사와 대화를 위한 공간으로, 가족 및 친구들이 모여 식사하며 소통하는 장소입니다. 파티나 모임에서 중심적인 역할을 하며, 식사의 분위기를 조성하는 중요한 요소입니다.\n\n#### 2. 음료수 (와인 잔)\n- **물체의 특징**: 와인 잔은 유리로 만들어져 있으며, 일반적으로 투명하고 우아한 디자인을 가지고 있습니다. 음료수의 색상은 다양할 수 있으나, 이 이미지에서는 붉은색으로 보입니다.\n- **물체의 예상되는 역할**: 와인 잔은 식사 중 음료를 제공하는 역할을 하며, 식사의 품격을 높이는 요소로 작용합니다. 또한, 축하의 의미를 담아 소통의 장을 마련하는 역할도 합니다.\n\n#### 3. 음식 (접시)\n- **물체의 특징**: 접시는 다양한 음식이 담겨져 있으며, 색상과 형태가 다양합니다. 주로 세라믹이나 유리로 만들어져 

### Speech TTS

In [19]:
import requests
import datetime
import os
import re
from dotenv import load_dotenv

load_dotenv()

tts_api_key = os.getenv("SPEECH_STUDIO_API_KEY")
tts_endpoint = os.getenv("TTS_KR_ENDPOINT")

def request_tts(text) :

    endpoint = tts_endpoint

    headers = {
        "Ocp-Apim-Subscription-Key" : tts_api_key,
        "Content-Type" : "application/ssml+xml",
        "X-Microsoft-OutputFormat" : "audio-16khz-128kbitrate-mono-mp3"
    }

    cleaned_text = re.sub(r'[^가-힣a-zA-Z0-9\s%,\.]', '', text)

    body = f"""
        <speak version='1.0' xml:lang='ko-KR'>
            <voice xml:lang='ko-KR' xml:gender='Male' name='ko-KR-InJoonNeural'>
                {cleaned_text}
            </voice>
        </speak>
    """

    response = requests.post(endpoint, headers=headers, data=body)

    if response.status_code != 200 :
        return None
    
    file_name = "tts_result_{}.wav".format(datetime.datetime.now().strftime("%Y%m%d_%H%M%S"))

    with open(file_name, "wb") as audio_file :
        audio_file.write(response.content)

    return file_name

request_tts("안녕하세요 만나서 반갑습니다.")

'tts_result_20251211_182600.wav'

### Gradio 화면

In [21]:
import datetime
import gradio as gr

with gr.Blocks() as demo :

    def stream_webcam(image_array) :
        result_image_array = detect_objects(image_array)
        return result_image_array
    
    def click_capture(image_array) :
        if image_array is None :
            return gr.Error("감지된 이미지가 없습니다.", duration=3)
        return image_array
    
    def click_send_gpt(image_array, histories) :

        content = request_gpt(image_array)
        label_text = datetime.datetime.now().strftime("%Y-%m-%d_%H:%M:%S")
        histories.append({
            "role" : "user", "content" : gr.Image(label=label_text, value=image_array)
        })
        histories.append({
            "role" : "assistant", "content" : content
        })

        return histories

    def change_chatbot(histories) :
        content = histories[-1]['content']
        file_name =  request_tts(content)
        return file_name


    with gr.Row() :
        webcam_image = gr.Image(label="실시간 화면", sources="webcam", mirror_webcam=False)
        output_image = gr.Image(label="감지 화면", interactive=False)
        capture_image = gr.Image(label="이상 징후 발생 화면", interactive=False)
    
    with gr.Row() :
        capture_button = gr.Button("이상 징후 발생")
        send_gpt_button = gr.Button("분석", )

    chatbot = gr.Chatbot(label="분석결과", type="messages", height=720)
    chatbot_audio = gr.Audio(label="분석 결과", interactive=False, autoplay=True)

    webcam_image.stream(stream_webcam, inputs=[webcam_image], outputs=[output_image])
    capture_button.click(click_capture, inputs=[output_image], outputs=[capture_image])
    send_gpt_button.click(click_send_gpt, inputs=[capture_image, chatbot], outputs=[chatbot])
    chatbot.change(change_chatbot, inputs=[chatbot], outputs=[chatbot_audio])

demo.launch()

c:\Users\EL91\AppData\Local\Programs\Python\Python311\Lib\site-packages\gradio\components\image.py:146: UserWarning: The `mirror_webcam` parameter is deprecated. Please use the `webcam_options` parameter with a `gr.WebcamOptions` instance instead.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7864
* To create a public link, set `share=True` in `launch()`.
